<head>
  <link href="https://fonts.cdnfonts.com/css/pf-tempesta-seven" rel="stylesheet">
  <link rel="stylesheet" href="https://use.typekit.net/bbp6pmz.css">
</head>

# NBA API Stats




In the following, we want to forecast the future success of potential NBA players and their performance in the NBA draft, as well as ascertain which colleges tend to produce the most successful NBA players. We analyze and evaluate player and game statistics to ascertain patterns and trends that contribute to successful NBA transitions.


In [1]:
# data processing
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from itertools import product
from lxml import html
import time
import requests
import json

# nba api stat endpoints
from nba_api.stats.static import teams
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import playerawards
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import leaguegamefinder

import circlify
import plotly.express as px
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as clr
%matplotlib inline
from pylab import *


<div class="mycode">

We will primarily be using `nba_api.stats.endpoints` for each API endpoint supported by [stats.nba.com](https://stats.nba.com/). For example, we use API endpoints such as `commonplayerinfo`, `drafthistory`

</div>


### Getting Team and Player IDs

The package also includes utilities for fetching player and team information available under `nba_api.stats.static`.

In [2]:
from nba_api.stats.static import teams

# get_teams returns a list of 30 dictionaries, each an NBA team
nba_teams = teams.get_teams()
print("Number of teams fetched: {}".format(len(nba_teams)))

Number of teams fetched: 30


In [3]:
from IPython.display import display, HTML
nba_teams_df = pd.DataFrame(nba_teams)

# Function to generate the URL based on team ID
def generate_team_logo_url(team_id):
    return f"https://cdn.nba.com/logos/nba/{team_id}/primary/L/logo.svg"

# Apply the function to create a new column with the logo URLs
nba_teams_df['logo_url'] = nba_teams_df['id'].apply(generate_team_logo_url)

# Function to display image in HTML format
def path_to_image_html(path):
    return f'<img src="{path}" width="50" >'

# Convert the DataFrame to HTML and display images
html_table = nba_teams_df.to_html(escape=False, formatters=dict(logo_url=path_to_image_html))
display(HTML(html_table))

,id,full_name,abbreviation,nickname,city,state,year_founded,logo_url
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949,
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946,
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970,
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002,
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966,
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980,
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976,
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946,
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967,
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970,


#### Get All NBA Players 

In [4]:
from nba_api.stats.static import players

# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
print(f"Number of players fetched: {len(nba_players)}")

nba_active_players = players.get_active_players()
print(f"Number of active players fetched: {len(nba_active_players)}")

Number of players fetched: 4900
Number of active players fetched: 531


----

## CommonTeamRoster

In [5]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import CommonTeamRoster

# get_teams returns a list of 30 dictionaries, each an NBA team
nba_teams = teams.get_teams()
all_nba_teams = [team["id"] for team in nba_teams]

In [6]:
%%script false

# List to hold data frames of all players info
all_players_info = []

# Loop through each team ID and get the roster data
for team_id in all_nba_teams:
    common_team_roster = CommonTeamRoster(
        team_id=team_id,
        league_id_nullable='00',  # NBA league ID
        season='2022-23',
        timeout=200
    )
    team_roster_df = common_team_roster.get_data_frames()[0]
    all_players_info.append(team_roster_df)

# Concatenate all individual team data frames into a single data frame
all_players_info_df = pd.concat(all_players_info, ignore_index=True)
all_players_info_df.to_csv("data/nba_players.csv")

Couldn't find program: 'false'


In [7]:
all_players_info_df = pd.read_csv("data/nba_players.csv")
all_players_info_df.head()

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612737,2022,0,Donovan Williams,Donovan,donovan-williams,NaN,G,6-6,190,"SEP 06, 2001",21.0,R,UNLV,1631495,NaN
1,1610612737,2022,0,Bruno Fernando,Bruno,bruno-fernando,0.0,F-C,6-10,240,"AUG 15, 1998",24.0,3,Maryland,1628981,Traded from HOU on 02/09/23
2,1610612737,2022,0,Jalen Johnson,Jalen,jalen-johnson,1.0,F,6-8,219,"DEC 18, 2001",21.0,1,Duke,1630552,#20 Pick in 2021 Draft
3,1610612737,2022,0,Trent Forrest,Trent,trent-forrest,2.0,G,6-4,210,"JUN 12, 1998",25.0,2,Florida State,1630235,Signed on 08/08/22
4,1610612737,2022,0,Aaron Holiday,Aaron,aaron-holiday,3.0,G,6-0,185,"SEP 30, 1996",26.0,4,UCLA,1628988,NaN


---

### Colleges

In [8]:
# Calculate the value counts and convert to dictionary
school_counts_dict = all_players_info_df['SCHOOL'].value_counts().to_dict()
school_counts_df = pd.DataFrame(list(school_counts_dict.items()), columns=['SCHOOL', 'Count'])

# Filter the DataFrame to include only rows with values > 1
filtered_school_counts_df = school_counts_df[school_counts_df['Count'] > 1]

In [9]:
def get_color(name, number):
    return list(sns.color_palette(palette=name, n_colors=number).as_hex())

# Create custom colormap and convert to a list of hex colors
cmap = clr.LinearSegmentedColormap.from_list('custom blue', ['#FDE3FE', '#E3DA82', '#FCA611', '#FB831E', '#FF4F2D', '#FA6094', '#0E2349'], N=86)
colors = [matplotlib.colors.rgb2hex(cmap(i)) for i in range(cmap.N)]

# Set the custom palette
custom_palette = sns.set_palette(sns.color_palette(colors))
pal_vi = get_color(custom_palette, len(filtered_school_counts_df))

In [10]:
import circlify

# Function to abbreviate 'State' to 'St.'
def abbreviate_state(school_name):
    return school_name.replace('State', 'St')
filtered_school_counts_df['SCHOOL'] = filtered_school_counts_df['SCHOOL'].apply(abbreviate_state)


# compute circle positions:
circles = circlify.circlify(
    filtered_school_counts_df['Count'].tolist(),
    target_enclosure=circlify.Circle(x=0, y=0, r=1),
    show_enclosure=False
)

bubble_df = pd.DataFrame({
    'x': [cir.x for cir in circles],
    'y': [cir.y for cir in circles],
    'r': [cir.r for cir in circles],
    'l': filtered_school_counts_df.sort_values('Count').SCHOOL.values,
    's': [math.pi * (cir.r ** 2) for cir in circles],
    'k': filtered_school_counts_df.sort_values('Count').Count.values
})


bubble_df["rank"] = bubble_df.sort_values(by="k").index
font_size = 90 * bubble_df.r.values
font_colors = ['white' if i > 9 else 'black' for i in bubble_df['k']]

C:\conda_tmp\ipykernel_43692\4198972769.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
import plotly.express as px

# Create the scatter plot
fig = px.scatter(bubble_df, x="x", y="y", custom_data=["l", "rank", "k"], color="rank", width=600, height=500,
                 size="s", hover_name="l", size_max=70, text="l",
                 color_continuous_scale=pal_vi, opacity=0.95)

# Update trace and layout settings
fig.update_traces(
    hovertemplate="%{customdata[0]}<br>%{customdata[2]}",
    texttemplate="<b style='letter-spacing: 0.25px;text-transform:uppercase;font-family:Roboto Condensed;'>%{customdata[0]}</b><br>%{customdata[2]}",
    textfont_color=font_colors,
    textfont_size=font_size
)

fig.update_layout(
    showlegend=False, coloraxis_showscale=False, xaxis_visible=False, yaxis_visible=False,
    yaxis_scaleanchor="x", yaxis_scaleratio=0.95, plot_bgcolor='rgba(0, 0, 0, 0)', paper_bgcolor='rgba(0, 0, 0, 0)',
    #title={'text': "<b>NBA Player Background</b>", 'y': 0.97, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top'},
    font=dict(
        family="Roboto",
        size=12,
        color="black"
    )
)

fig.show()